# 신경망 모델 구성하기
* 신경망은 데이터에 대한 연산을 수행하는 Layer/Module로 구성되어 있습니다.
* <code>torch.nn</code> 네임스페이스는 신경망을 구성하는데 필요한 모든 구성요소를 제공한다.
* Pytorch의 모든 모듈은 <code>nn.Module</code>의 하위 클래스(subclass)이다.
* 신경망은 다른 모듈로 구성된 모듈(계층/Layer)이다.
* 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있다.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    'cuda' 
    if torch.cuda.is_available() 
    else 'mps' 
    if torch.backends.mps.is_available() 
    else 'cpu'
)

print(f'Using {device} device')

Using cpu device


# 클래스 정의하기
* 신경망 모델을 <code>nn.Module</code>의 하위 클래스로 정의하고, <code>\_\_init\_\_</code>에서 신경망 계층들을 초기화한다.
* <code>nn.Module</code>을 상속받은 모든 클래스는 <code>forward</code>메소드에 입력 데이터에 대한 연산들을 구현한다.
## 궁금한 점
* model을 print 했을 때, <code>\_\_init\_\_</code>에 따라 출력된다.
* 근데 <code>forward</code>에서 연산 순서를 다르게 하는 건 영향이 없는 건가
* 제일 영향력 있어 보이는 건 <code>forward</code>인데?

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

* <code>NeuralNetwork</code>의 인스턴스를 생성하고, device로 이동한 뒤, 구조를 출력한다.

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device) # cpu로 할당
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Prdicted Class: {y_pred}')

Prdicted Class: tensor([0])


# Model Layer
* FashionMNIST 모델의 계층들을 살펴보자.
* 이를 설명하기 위해 28x28 크기의 mini-batch 사이즈가 3인 mini-batch를 가져와서 신경망을 통과할 때 어떤 일이 발생하는지 알아보자.

In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


# 1) nn.Flatten
<code>nn.Flatten</code> 계층을 초기화하여 각 28x28 2D이미지를 784 픽셀 값을 갖는 연속된 배열로 변환한다. (dim = 0의 미니배치 차원은 유지)

In [8]:
# nn.Flatten() 같은 건 뭐라 해야 하지?
# Object를 가져온 거 같은데 바로 메소드처럼 쓸 수 있다니

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# 2) nn.Linear
* <code>nn.Linear</code>는 저장된 weight와 bias를 사용하여 입력에 linear transformation을 적용하는 모듈이다.

In [9]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# 3) nn.ReLU
* 비선형 활성화(activation)은 모델의 입력과 출력 사이에 복잡한 mapping을 만든다.
* 비선형 활성화는 선형 변환 후에 적용되어 <b>비선형성</b>을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.
* 이 모델에서는 <code>nn.ReLU</code>를 선형 layter들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 activation을 도입할 수도 있다.

In [10]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU: {hidden1}')

Before ReLU: tensor([[ 0.1407,  0.2383, -0.0635, -0.0060,  0.2283,  0.2278,  0.1435, -0.2386,
          0.0178,  0.2789,  0.3112,  0.1029,  0.3116, -0.0691, -0.5604, -0.6370,
         -0.7093, -0.0451,  0.4186,  0.5203],
        [-0.1895,  0.3730, -0.2596, -0.0122,  0.3311,  0.4573,  0.1157,  0.1438,
          0.3303, -0.0767,  0.3975,  0.1567,  0.1277, -0.2094, -0.0760, -0.3582,
         -0.5674,  0.1475,  0.0263,  0.2786],
        [-0.0999,  0.2995, -0.4041, -0.2248,  0.0017,  0.2850, -0.0065,  0.0072,
         -0.0250, -0.1937,  0.2294,  0.1579,  0.1761, -0.3746, -0.1298, -0.2677,
         -0.4355,  0.0110,  0.1522, -0.1370]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1407, 0.2383, 0.0000, 0.0000, 0.2283, 0.2278, 0.1435, 0.0000, 0.0178,
         0.2789, 0.3112, 0.1029, 0.3116, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.4186, 0.5203],
        [0.0000, 0.3730, 0.0000, 0.0000, 0.3311, 0.4573, 0.1157, 0.1438, 0.3303,
         0.0000, 0.3975, 0.1567, 0.1277, 0.0000, 0.00

# 4) nn.Sequential
* <code>nn.Sequential</code>은 순서를 갖는 모듈의 컨테이너이다.
* 데이터는 정의된 것과 같은 순서로 모든 모듈을 통해 전달된다.
* 순차 컨테이너를 사용하여 <code>seq_modules</code>와 같은 신경망을 빠르게 만들 수 있다.

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

# 5) nn.Softmax
* 신경망의 마지막 선형 계층은 <code>nn.Softmax</code> 모듈에 전달될 ([-inf, inf] 범위의 raw value인) logits를 반환한다.
* logits는 모델의 각 classification에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 scaling된다. 
* dim 매개변수는 값의 합이 1이 되는 차원을 나타낸다.

In [12]:
softmax = nn.Softmax(dim=1)
pre_probab = softmax(logits)

In [13]:
# mini-batch 사이즈가 3에 따라 한 이미지에 대해 10개의 class로 classification

pre_probab

tensor([[0.0844, 0.0957, 0.1371, 0.1107, 0.0765, 0.1157, 0.1234, 0.0841, 0.0948,
         0.0775],
        [0.0825, 0.0941, 0.1277, 0.1120, 0.0825, 0.1103, 0.1221, 0.0848, 0.0951,
         0.0889],
        [0.0829, 0.0881, 0.1252, 0.1241, 0.0782, 0.1229, 0.1202, 0.0855, 0.0967,
         0.0761]], grad_fn=<SoftmaxBackward0>)

# Model Parameter
* 신경망 내부의 많은 layer들은 Parameterized된다.
* 즉, 학습 중에 최적화되는 weight와 bias와 연관지어진다.
* <code>nn.Module</code>을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 <code>parameters()</code> 및 <code>named_parameters()</code> 메소드로 모든 매개변수에 접근할 수 있게 된다.
* 이 예제에서는 각 매개변수들을 iterate하며, 매개변수의 크기와 값을 출력한다.

In [15]:
print(f'Model Structure {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param}\n')

Model Structure NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: Parameter containing:
tensor([[ 0.0345,  0.0326,  0.0338,  ...,  0.0025,  0.0337,  0.0040],
        [ 0.0077, -0.0174,  0.0108,  ...,  0.0158,  0.0011,  0.0227],
        [ 0.0131, -0.0330, -0.0157,  ..., -0.0328,  0.0106, -0.0068],
        ...,
        [ 0.0073, -0.0235,  0.0047,  ...,  0.0044, -0.0021,  0.0311],
        [-0.0263, -0.0186, -0.0233,  ...,  0.0301,  0.0006,  0.0253],
        [ 0.0087, -0.0349,  0.0250,  ..., -0.0357,  0.0217, -0.0259]],
       requires_grad=True)

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: Parameter containing:
tensor([-0.010